In [149]:
import pandas as pd 
import numpy as np

In [150]:
df_germany = pd.read_excel('../data /imf_GERMANY_data.xlsx')

# Load the second XLSX file into a DataFrame
df_us = pd.read_excel('../data /imf_US_data.xlsx')

In [151]:
df_germany.rename(columns={'International Financial Statistics (IFS)': 'Month'}, inplace=True)

# Rename the unnamed columns based on the provided names
new_column_names = [
    'Month',
    'Economic Activity, Industrial Production, Index',
    'Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate',
    'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar',
    'Prices, Consumer Price Index, All items, Index'
]
df_germany.columns = new_column_names
df_germany = df_germany.drop([0, 1])

In [152]:
df_us.rename(columns={'International Financial Statistics (IFS)': 'Month'}, inplace=True)

# Rename the unnamed columns based on the provided names
new_column_names1 = [
    'Month',
    'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar',
    'Prices, Consumer Price Index, All items, Index'
]

df_us.columns = new_column_names1
df_us = df_us.drop([0, 1,])

df_germany['Country'] = 'Germany'
df_us['Country'] = 'US'

# Merge the two DataFrames
merged_df = pd.concat([df_germany, df_us], ignore_index=True)


In [153]:
columns = ['Month', 'Country', 'Economic Activity, Industrial Production, Index',
           'Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate',
           'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar',
           'Prices, Consumer Price Index, All items, Index']

merged_df = merged_df[columns]


In [154]:
df_germany = merged_df[merged_df['Country'] == 'Germany']
df_us = merged_df[merged_df['Country'] == 'US']

# Reset index for both DataFrames
df_germany.reset_index(drop=True, inplace=True)
df_us.reset_index(drop=True, inplace=True)

# Calculate the ratio of US CPI to Germany CPI
cpi_ratio = df_us['Prices, Consumer Price Index, All items, Index'].values / df_germany['Prices, Consumer Price Index, All items, Index'].values

# Calculate the monthly growth in the nominal exchange rate for Germany
df_germany['Real Exchange Rate'] = cpi_ratio * df_germany['Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate']

# Add the new variable to the original merged DataFrame
merged_df.loc[merged_df['Country'] == 'Germany', 'Real Exchange Rate'] = df_germany['Real Exchange Rate']

# Rename the column
merged_df.rename(columns={'Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate': 'Nominal Exchange Rate'}, inplace=True)



/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/3081440155.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_germany['Real Exchange Rate'] = cpi_ratio * df_germany['Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate']


In [155]:
# Calculate the percentage change in the nominal exchange rate
merged_df['Nominal Exchange Rate Growth (%)'] = merged_df['Nominal Exchange Rate'].pct_change() * 100

# Calculate the percentage change in the real exchange rate
merged_df['Real Exchange Rate Growth (%)'] = merged_df['Real Exchange Rate'].pct_change() * 100

# Display the first 10 rows with relevant columns
print(merged_df[['Month', 'Country', 'Real Exchange Rate', 'Real Exchange Rate Growth (%)']].head(10))



      Month  Country Real Exchange Rate  Real Exchange Rate Growth (%)
0  Dec 1959  Germany           2.300359                            NaN
1  Jan 1960  Germany           2.292535                      -0.340136
2  Feb 1960  Germany           2.313504                       0.914640
3  Mar 1960  Germany           2.313504                       0.000000
4  Apr 1960  Germany           2.308184                      -0.229943
5  May 1960  Germany           2.301645                      -0.283291
6  Jun 1960  Germany           2.309447                       0.338983
7  Jul 1960  Germany           2.309447                       0.000000
8  Aug 1960  Germany           2.309447                       0.000000
9  Sep 1960  Germany           2.316008                       0.284096


/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/650098298.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  merged_df['Nominal Exchange Rate Growth (%)'] = merged_df['Nominal Exchange Rate'].pct_change() * 100
/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/650098298.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['Nominal Exchange Rate Growth (%)'] = merged_df['Nominal Exchange Rate'].pct_change() * 100
/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/650098298.py:5: FutureWarning: The default fill_method='pa

In [156]:
# Find the Real Exchange Rate for December 1990
rer_dec_1990 = merged_df.loc[merged_df['Month'] == 'Dec 1990', 'Real Exchange Rate'].values[0]

# Construct the Index
merged_df['RER_Index'] = merged_df['Real Exchange Rate'] / rer_dec_1990


In [157]:
# Convert 'Month' column to datetime format for proper sorting
merged_df['Month'] = pd.to_datetime(merged_df['Month'], format='%b %Y')

# Sort by Country first, then by Month (chronologically)
merged_df = merged_df.sort_values(by=['Country', 'Month'])

# Compute Monthly Inflation Rate within each country
merged_df['Monthly Inflation Growth (%)'] = merged_df.groupby('Country')[
    'Prices, Consumer Price Index, All items, Index'].pct_change() * 100

# Convert 'Month' back to string format if needed for display
merged_df['Month'] = merged_df['Month'].dt.strftime('%b %Y')

# Calculate the percentage change in industrial production
merged_df['Monthly Industrial Production Growth (%)'] = merged_df['Economic Activity, Industrial Production, Index'].pct_change() * 100

/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/299475220.py:15: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  merged_df['Monthly Industrial Production Growth (%)'] = merged_df['Economic Activity, Industrial Production, Index'].pct_change() * 100
/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/299475220.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['Monthly Industrial Production Growth (%)'] = merged_df['Economic Activity, Industrial Production, Index'].pct_change() * 100


In [158]:
# Calculate the year-over-year (YoY) growth in industrial production
merged_df['Industrial Production YoY Growth (%)'] = merged_df['Economic Activity, Industrial Production, Index'].pct_change(periods=12) * 100



/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/2397368808.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  merged_df['Industrial Production YoY Growth (%)'] = merged_df['Economic Activity, Industrial Production, Index'].pct_change(periods=12) * 100
/var/folders/16/ybdglc7d23vfl64gx8z5bp6m0000gn/T/ipykernel_79693/2397368808.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['Industrial Production YoY Growth (%)'] = merged_df['Economic Activity, Industrial Production, Index'].pct_change(periods=12) * 100


In [159]:
# Identify the value of international reserves in January 1960
base_value = merged_df.loc[merged_df['Month'] == 'Jan 1960', 'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].values[0]

# Compute the index (value relative to January 1960)
merged_df['International Reserves Index'] = (merged_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] / base_value) * 100

In [160]:
# List of columns to drop
columns_to_drop = [
    'Economic Activity, Industrial Production, Index',
    'Nominal Exchange Rate',
    'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar',
    'Prices, Consumer Price Index, All items, Index',
    'Real Exchange Rate',
]

# Drop the columns from the DataFrame
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')


In [161]:
import pandas as pd

# Set display options to limit the number of rows and columns shown
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)

# Now, when you print the DataFrame, it will be limited to the specified number of rows and columns
print(merged_df)

        Month  Country  Nominal Exchange Rate Growth (%)  \
0    Dec 1959  Germany                               NaN   
1    Jan 1960  Germany                               0.0   
2    Feb 1960  Germany                               0.0   
3    Mar 1960  Germany                               0.0   
4    Apr 1960  Germany                               0.0   
..        ...      ...                               ...   
741  Aug 1990       US                               0.0   
742  Sep 1990       US                               0.0   
743  Oct 1990       US                               0.0   
744  Nov 1990       US                               0.0   
745  Dec 1990       US                               0.0   

     Real Exchange Rate Growth (%) RER_Index Monthly Inflation Growth (%)  \
0                              NaN  1.713301                          NaN   
1                        -0.340136  1.707474                          0.0   
2                         0.914640  1.723091    

In [162]:
merged_df

,Month,Country,Nominal Exchange Rate Growth (%),Real Exchange Rate Growth (%),RER_Index,Monthly Inflation Growth (%),Monthly Industrial Production Growth (%),Industrial Production YoY Growth (%),International Reserves Index
0,Dec 1959,Germany,NaN,NaN,1.713301,NaN,NaN,NaN,101.848342
1,Jan 1960,Germany,0.0,-0.340136,1.707474,0.0,-4.019729,NaN,100.0
2,Feb 1960,Germany,0.0,0.914640,1.723091,-0.568147,-0.488181,NaN,101.740143
3,Mar 1960,Germany,0.0,0.000000,1.723091,0.0,3.743868,NaN,105.128964
4,Apr 1960,Germany,0.0,-0.229943,1.719129,0.571393,6.470881,NaN,110.837171
...,...,...,...,...,...,...,...,...,...
741,Aug 1990,US,0.0,0.000000,NaN,0.920245,0.000000,0.0,1670.326762
742,Sep 1990,US,0.0,0.000000,NaN,0.835866,0.000000,0.0,1693.935801
743,Oct 1990,US,0.0,0.000000,NaN,0.602864,0.000000,0.0,1753.798992
744,Nov 1990,US,0.0,0.000000,NaN,0.224719,0.000000,0.0,1758.185093


In [163]:
# Define a function to detect strict (fewer) outliers
def detect_outliers_strict(df, column, iqr_threshold=3.0, z_threshold=4.0):  
    # Compute IQR bounds
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound_iqr = Q1 - iqr_threshold * IQR
    upper_bound_iqr = Q3 + iqr_threshold * IQR
    
    # Compute Z-score bounds
    mean = df[column].mean()
    std_dev = df[column].std()
    lower_bound_z = mean - z_threshold * std_dev
    upper_bound_z = mean + z_threshold * std_dev

    # Ensure fewer values are flagged (outliers must violate both conditions)
    return (df[column] < lower_bound_iqr) & (df[column] < lower_bound_z) | \
           (df[column] > upper_bound_iqr) & (df[column] > upper_bound_z)

# Apply function to numeric columns
numeric_columns = merged_df.select_dtypes(include=[np.number]).columns
outlier_locations = []

for col in numeric_columns:
    outliers = detect_outliers_strict(merged_df, col, iqr_threshold=3.0, z_threshold=4.0)
    
    # Store row index, month, column, and outlier value
    for index in merged_df[outliers].index:
        outlier_locations.append((index, merged_df.loc[index, "Month"], col, merged_df.loc[index, col]))

# Convert outlier locations to a DataFrame for easier viewing
outliers_df = pd.DataFrame(outlier_locations, columns=["Row Index", "Month", "Column", "Outlier Value"])

# Display detected outliers with the Month column
print("Detected Outliers with Month Information:")
print(outliers_df)


Detected Outliers with Month Information:
    Row Index     Month                                Column  Outlier Value
0         119  Nov 1969      Nominal Exchange Rate Growth (%)      -8.500000
1         145  Jan 1972      Nominal Exchange Rate Growth (%)      -6.857094
2         162  Jun 1973      Nominal Exchange Rate Growth (%)      -7.669997
3         163  Jul 1973      Nominal Exchange Rate Growth (%)      -9.474501
4         167  Nov 1973      Nominal Exchange Rate Growth (%)       6.913703
5         254  Feb 1981      Nominal Exchange Rate Growth (%)       6.774258
6         304  Apr 1985      Nominal Exchange Rate Growth (%)      -6.835489
7         310  Oct 1985      Nominal Exchange Rate Growth (%)      -6.890602
8         325  Jan 1987      Nominal Exchange Rate Growth (%)      -6.703153
9         335  Nov 1987      Nominal Exchange Rate Growth (%)      -6.708840
10        119  Nov 1969         Real Exchange Rate Growth (%)      -8.217983
11        145  Jan 1972         Re